# Stable Diffusion Outpainting

In [ ]:
!pip install diffusers transformers accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 23.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylin

In [ ]:
import PIL
import requests
import torch
from io import BytesIO
from PIL import Image
from diffusers import StableDiffusionInpaintPipeline
import numpy as np

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


In [ ]:
!pip install perlin_noise

In [ ]:
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting"
).to('cuda')
pipe.enable_attention_slicing()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors not found


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

...

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/335M [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
from PIL import Image
import numpy as np
import random
from perlin_noise import PerlinNoise
from sklearn.preprocessing import minmax_scale
import click

# Параметры
downscale_factor = 1.5
image_size = 512

# Инициализация генераторов шума Perlin
noise11 = PerlinNoise(octaves=10)
noise12 = PerlinNoise(octaves=5)
noise21 = PerlinNoise(octaves=10)
noise22 = PerlinNoise(octaves=5)
noise31 = PerlinNoise(octaves=10)
noise32 = PerlinNoise(octaves=5)

def noise_mult_1(i, j, xpix=image_size, ypix=image_size):
    return noise11([i / xpix, j / ypix]) + 0.5 * noise12([i / xpix, j / ypix])

def noise_mult_2(i, j, xpix=image_size, ypix=image_size):
    return noise21([i / xpix, j / ypix]) + 0.5 * noise22([i / xpix, j / ypix])

def noise_mult_3(i, j, xpix=image_size, ypix=image_size):
    return noise31([i / xpix, j / ypix]) + 0.5 * noise32([i / xpix, j / ypix])

def get_mask_image(img, downscale_factor=2):
    img_downscaled = img.resize((int(img.size[0] / downscale_factor), int(img.size[1] / downscale_factor)))
    img_arr = np.array(img_downscaled)

    # Создание белой маски
    full_mask = np.ones((image_size, image_size, 3), dtype=np.uint8) * 255

    # Случайный выбор позиции для вставки уменьшенного изображения
    x_offset = random.randint(0, image_size - img_downscaled.size[0])
    y_offset = random.randint(0, image_size - img_downscaled.size[1])

    for i in range(img_downscaled.size[1]):
        for j in range(img_downscaled.size[0]):
            full_mask[y_offset + i, x_offset + j] = 0  # Черные пиксели на маске

    return full_mask, img_downscaled, x_offset, y_offset

def get_init_image(noised_img, full_mask, x_offset, y_offset, xpix=image_size, ypix=image_size):
    click.echo('Generating noise ...')
    pic = [[[noise_mult_1(i, j), noise_mult_2(i, j), noise_mult_3(i, j)] for j in range(xpix)] for i in range(ypix)]
    click.echo('Noise generated!')

    scaled_noise = minmax_scale(np.array(pic).flatten(), (0, 255)).reshape((xpix, ypix, 3))
    scaled_noise = scaled_noise.astype(np.uint8)

    init_image = scaled_noise.copy()
    noised_img_arr = np.array(noised_img)

    for i in range(noised_img_arr.shape[0]):
        for j in range(noised_img_arr.shape[1]):
            init_image[y_offset + i, x_offset + j] = noised_img_arr[i, j]

    return init_image

def get_init_mask_image(img, downscale_factor=2):
    full_mask, noised_image, x_offset, y_offset = get_mask_image(img, downscale_factor)
    init_image = get_init_image(noised_image, full_mask, x_offset, y_offset)
    return init_image, full_mask


In [ ]:
from google.colab import drive
#подключаем гугл диск
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os
from PIL import Image

# Путь к ZIP файлу
zip_file = '/content/drive/MyDrive/NIRM2/sd_outpainting/init_image/outpainting_init_image.zip'

# Директория для распаковки изображений
extract_dir = '/content/init_image'

# Создаем директорию для извлеченных изображений, если она не существует
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

# Распаковываем архив
with zipfile.ZipFile(zip_file, 'r') as z:
    z.extractall(extract_dir)

# Получаем список файлов в директории с изображениями
image_files = os.listdir(extract_dir)


In [ ]:
from PIL import Image
import numpy as np
import random
from perlin_noise import PerlinNoise
from sklearn.preprocessing import minmax_scale
import os
import click

# Параметры
downscale_factor = 1.5
image_size = 512

# Инициализация генераторов шума Perlin
noise11 = PerlinNoise(octaves=10)
noise12 = PerlinNoise(octaves=5)
noise21 = PerlinNoise(octaves=10)
noise22 = PerlinNoise(octaves=5)
noise31 = PerlinNoise(octaves=10)
noise32 = PerlinNoise(octaves=5)

def noise_mult_1(i, j, xpix=image_size, ypix=image_size):
    return noise11([i / xpix, j / ypix]) + 0.5 * noise12([i / xpix, j / ypix])

def noise_mult_2(i, j, xpix=image_size, ypix=image_size):
    return noise21([i / xpix, j / ypix]) + 0.5 * noise22([i / xpix, j / ypix])

def noise_mult_3(i, j, xpix=image_size, ypix=image_size):
    return noise31([i / xpix, j / ypix]) + 0.5 * noise32([i / xpix, j / ypix])

def get_mask_image(img, downscale_factor=2):
    img_downscaled = img.resize((int(img.size[0] / downscale_factor), int(img.size[1] / downscale_factor)))
    img_arr = np.array(img_downscaled)

    # Создание белой маски
    full_mask = np.ones((image_size, image_size, 3), dtype=np.uint8) * 255

    # Случайный выбор позиции для вставки уменьшенного изображения
    x_offset = random.randint(0, image_size - img_downscaled.size[0])
    y_offset = random.randint(0, image_size - img_downscaled.size[1])

    for i in range(img_downscaled.size[1]):
        for j in range(img_downscaled.size[0]):
            full_mask[y_offset + i, x_offset + j] = 0  # Черные пиксели на маске

    return full_mask, img_downscaled, x_offset, y_offset

def get_init_image(noised_img, full_mask, x_offset, y_offset, xpix=image_size, ypix=image_size):
    click.echo('Generating noise ...')
    pic = [[[noise_mult_1(i, j), noise_mult_2(i, j), noise_mult_3(i, j)] for j in range(xpix)] for i in range(ypix)]
    click.echo('Noise generated!')

    scaled_noise = minmax_scale(np.array(pic).flatten(), (0, 255)).reshape((xpix, ypix, 3))
    scaled_noise = scaled_noise.astype(np.uint8)

    init_image = scaled_noise.copy()
    noised_img_arr = np.array(noised_img)

    for i in range(noised_img_arr.shape[0]):
        for j in range(noised_img_arr.shape[1]):
            init_image[y_offset + i, x_offset + j] = noised_img_arr[i, j]

    return init_image

def get_init_mask_image(img, downscale_factor=2):
    full_mask, noised_image, x_offset, y_offset = get_mask_image(img, downscale_factor)
    init_image = get_init_image(noised_image, full_mask, x_offset, y_offset)
    return init_image, full_mask


# Получаем список файлов в директории с изображениями
image_files = os.listdir('/content/test')
NB_STEPS = 3
base_image = Image.open('/content/test/000000001584.jpg').resize((512,512))

for i in range(NB_STEPS):
    print('STEP: ', i)
    init_image, mask_image = get_init_mask_image(base_image)
    init_image_pil = Image.fromarray(init_image)
    mask_image_pil = Image.fromarray(mask_image)
    init_image_pil.save('/content/init.jpg')

    # Outpainting with stable diffusion pipeline
    outpainted_image = pipe(prompt="the bus is driving along the road", image=init_image_pil, mask_image=mask_image_pil).images[0]

    # Saving images
    mask_image_pil.save(f'/content/drive/MyDrive/NIRM2/sd_outpainting/mask_image/mask_{i}.jpg')
    outpainted_image.save(f'/content/drive/MyDrive/NIRM2/sd_outpainting/output_image/{i}.jpg')
    print(f"Saved image /content/drive/MyDrive/NIRM2/sd_outpainting/output_image/{i}.jpg")


STEP:  0
Generating noise ...
Noise generated!


  0%|          | 0/50 [00:00<?, ?it/s]

Saved image /content/drive/MyDrive/NIRM2/sd_outpainting/output_image/0.jpg
STEP:  1
Generating noise ...
Noise generated!


  0%|          | 0/50 [00:00<?, ?it/s]

Saved image /content/drive/MyDrive/NIRM2/sd_outpainting/output_image/1.jpg
STEP:  2
Generating noise ...
Noise generated!


  0%|          | 0/50 [00:00<?, ?it/s]

Saved image /content/drive/MyDrive/NIRM2/sd_outpainting/output_image/2.jpg
